In [1]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit

In [2]:
pdb = app.PDBFile('data/ethane.pdb')
forcefield = app.ForceField('data/ethane.gaff2.xml')

In [3]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [4]:
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

In [5]:
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

In [6]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

Minimizing...
Potential energy before minimization is 4.467818224810632 kJ/mol
Potential energy after minimization is 4.389967678922688 kJ/mol


In [7]:
from sys import stdout

print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(2500)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,17.467083564535933,312.00174943104463
200,13.424112014181595,183.17643718055925
300,18.236033759740543,293.0589592182455
400,15.195366748750512,293.61503988319663
500,16.84407004879646,231.57092263812788
600,32.944864512186,295.9339867619131
700,34.129648269213895,447.083117538121
800,19.572302283649996,413.6581285594615
900,20.60133124617421,340.1743284472552
1000,17.364480983713786,361.38564446837677
1100,17.508714679030135,283.8591417481635
1200,10.507150978716005,250.11108411782828
1300,28.0154138046385,326.9028813708034
1400,21.277989139320955,202.03889599048367
1500,25.568117316473348,226.35599045797716
1600,28.282438918366413,150.2286136474186
1700,14.947898349715802,268.6343678221977
1800,17.827337438698386,227.18210824899003
1900,10.796233855577377,209.44225208626148
2000,22.29338512573151,225.85600837893153
2100,23.407368674454332,182.20058921715616
2200,27.721936733306574,228.05553198606378
2300,23.7

In [8]:
import time as time

print('Running Production...')

# Begin timer
tinit=time.time()

# Clear simulation reporters
simulation.reporters.clear()

# Reinitialize simulation reporters. We do this because we want different information printed from the production run than the equilibration run.
# output basic simulation information below every 250000 steps - (which is equal to 2 fs(250,000) = 500,000 fs = 500 ps)
simulation.reporters.append(app.StateDataReporter(stdout, 250000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))

# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps - 0.2 ps
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

# run the simulation for 1.0x10^7 steps - 20 ns
simulation.step(10000000)

# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Running Production...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
250000,500.0000000016593,21.121519160194445,576.983271586312,0
500000,999.9999999901769,31.816912631010005,636.9383284391093,2.51e+04
750000,1499.9999999783536,21.355562914396536,527.4501791727316,2.52e+04
1000000,1999.9999999665301,24.127645706202777,153.43402010333287,2.52e+04
1250000,2499.9999999547067,28.15865429333668,307.5805675986104,2.52e+04
1500000,2999.9999999428833,18.662677527706325,239.12679279133303,2.52e+04
1750000,3499.99999993106,13.966811421504847,199.54801595317795,2.52e+04
2000000,3999.9999999192364,21.35260217054877,179.41548057328072,2.52e+04
2250000,4499.9999999992715,13.216315675568104,548.1976259772604,2.52e+04
2500000,5000.000000101135,23.860156111361228,285.26476882779565,2.52e+04
2750000,5500.000000202998,14.577097397731814,298.03697962582066,2.52e+04
3000000,6000.000000304862,19.35568711748448,222.47642264899213,2.52e+04
3250000,6500.000000406725,19.668

In [10]:
import mdtraj as md

traj = md.load('ethane_sim.dcd', top='data/ethane.pdb')

import nglview as ngl

visualize = ngl.show_mdtraj(traj)
visualize

NGLWidget(max_frame=99999)

In [11]:
ls

alkane-simulation.ipynb   data/                     ethane_sim.dcd
buckminsterfullerene.xyz  environment.yml


In [12]:
vis = ngl.show_openbabel('buckminsterfullerene.xyz')

ModuleNotFoundError: No module named 'openbabel'